## Pip installs

In [1]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install pysam

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install HTSeq

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install enformer-pytorch>=0.5

In [7]:
!pip install polars

Defaulting to user installation because normal site-packages is not writeable


## Load modules and variable declarations

In [1]:
from dnaDataSet import dnaDataSet

/home/shawn/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import polars as pl
from enformer_pytorch import Enformer, GenomeIntervalDataset
from datasets import concatenate_datasets, load_dataset
import os
import numpy as np
import pandas as pd
import pysam
import json 
import pickle

In [3]:
from llama_cpp import Llama

In [21]:
#can change the model path to any quantized model binary
#TODO - make a script version of this with option for model path, relative context length, tsv_path, bam_path
modelPath="/home/shawn/Programming/ai_stuff/llama.cpp/models/30B/ggml-model-q4_0.bin" 
memoryDir="/home/shawn/datasets/llm_memory"

In [22]:
tsv_dir= "/home/shawn/datasets/enformer_data"


In [23]:
#good testing but use a gvcf instead containing all known mutations first
tsv_path = tsv_dir + "/CARTPt04_Scan2_svc_merged_extract_snp.hg38_multianno.tsv"

In [24]:
bam_path=tsv_dir + '/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-D4_S26.realigned_deduped_sorted.bam'

## test stuff with the class

In [25]:
dnaset = dnaDataSet()

In [26]:
dnaset

dnaDataSet object
 current model being used: False,
 mutationDictionary: {},
 bamsDictionary: {},
 tsv: Empty DataFrame
Columns: []
Index: [],
 relativeContextLength: 4,
 memoryDir: /home/shawn/Programming/ai_stuff/dna-llama,
 promptOutput: 

In [27]:
dnaset2=dnaDataSet(modelPath=modelPath, memoryDir=memoryDir)

## Few shot learning

In [28]:
'''
    Idea was to finetune with the dataset form makeLlamaDataset, but for now we are just trying to use it to do few shot learning by taking some examples
    from it and using it to get it to tell you the mutated base in a read you give it
'''

'\n    Idea was to finetune with the dataset form makeLlamaDataset, but for now we are just trying to use it to do few shot learning by taking some examples\n    from it and using it to get it to tell you the mutated base in a read you give it\n'

In [29]:
bam_path=tsv_dir + '/' + 'CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam'
bed_path=tsv_dir + '/' + '/home/shawn/datasets/enformer_data/Homo_sapiens_assembly38_n25chr.bed'

In [30]:
dnaset2.makeLlamaDataset(tsv_dir, bam_path, bed_path)

the tsv file is: /home/shawn/datasets/enformer_data/01_final_clonal_somatic_calls.tsv
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A8_S50 0/1
C
LOC729737
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B4_S24 0/1
C
LOC729737
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51 0/1
C
LOC729737
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E10_S68 0/1
C
LOC729737
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E11_S75 0/1
C
LOC729737


[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is old

139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5 0/1
C
LOC729737
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F11_S76 0/1
C
LOC729737
139836
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F6_S41 0/1
C
LOC729737
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A8_S50 0/1
A
NOTCH2NLR
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B4_S24 0/1
A
NOTCH2NLR


[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is old

120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51 0/1
A
NOTCH2NLR
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E10_S68 0/1
A
NOTCH2NLR
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F10_S69 0/1
A
NOTCH2NLR
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F11_S76 1/1
A
NOTCH2NLR
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F1_S6 0/1
A
NOTCH2NLR
120723816
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F6_S41 0/1
A
NOTCH2NLR


...

146983217
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F11_S76 0/1
T
NBPF12
146983217
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F1_S6 1/1
T
NBPF12
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A8_S50 0/1
T
PDE4DIPP1
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B4_S24 0/1
T
PDE4DIPP1
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51 0/1
T
PDE4DIPP1
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E11_S75 0/1
T
PDE4DIPP1


[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is old

148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5 0/1
T
PDE4DIPP1
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F11_S76 0/1
T
PDE4DIPP1
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F1_S6 0/1
T
PDE4DIPP1
148086624
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-F6_S41 0/1
T
PDE4DIPP1
201369845
CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51 0/1
C
TNNT2
the start pos from tsv is 201369845 the start pos from pileup is 201369750 the the gene is: TNNT2 the read is: ['0', '201369749', '1\nVH00736:15:AAAVYTKHV:2:1611:14084:46302', '163', '#0', '201369750', '60', '100M', '#0', '201369964', '314', 'GGACAAGGAGGCTGCACTCGGGACAGCAGGCAGCAGAAAGCACCACAGAAGGAAAGGCTGTACTACCGTCTTCGTCCTCTCTCCAGTCCTCCTCTTCTGA', "array('B', [28, 30, 28, 27, 26, 27, 29, 29, 30, 31, 30, 31, 31, 32, 31, 30, 30, 31, 32, 25, 31, 31, 31, 30, 30, 32, 31, 30, 31, 31, 31, 30, 32, 31, 30, 32, 31, 31, 31, 32, 31, 30, 30, 30, 28, 30, 30, 32, 31, 31, 32, 31, 31, 31, 31, 32, 31, 30, 31, 32, 8, 30, 30, 31, 30, 30, 30, 25, 

[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /home/shawn/datasets/enformer_data/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-E1_S5.recalibrated_realigned_deduped_sorted.bam.bai


{'instruction': 'This read has a mutation. What is the metadata for this read? The read: GTCCTCCTCTTCTGAGGTTCAGGGAGTGGCCGCAGCGGAGGGGAAAAGCGAGACAGGTTAGTCTGGGAGCAGAGAGCCCGTGGCAGGAGCAGCCACCCAG',
 'input': 'Chromsome position: chr1, Start position: 201369845, Reference Genome: hg38',
 'output': 'the start pos from tsv is 201369845 the start pos from pileup is 201369835 the the gene is: TNNT2, the mutated base is: A'}

In [33]:
dnaset2.fewShotLearning('TAGAAAATGTGGATGAGACCTTCTGCATAGATAACGAAGCGCTATATGACATATGTTCCAGGACCCTAAAACTGCCCACACCCACCTATGGTGACCTGAA', quantized=True)

llama.cpp: loading model from /home/shawn/Programming/ai_stuff/llama.cpp/models/30B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 6656
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 52
llama_model_load_internal: n_layer    = 60
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 3 (mostly Q4_1)
llama_model_load_internal: n_ff       = 17920
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 30B
llama_model_load_internal: ggml ctx size = 110.30 KB
llama_model_load_internal: mem required  = 25573.12 MB (+ 3124.00 MB per state)
AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
llama_init_from_file: kv self size  =  780.00 MB


{'id': 'cmpl-b1c98ede-8c88-4014-a215-850bdfaf8122', 'object': 'text_completion', 'created': 1682400420, 'model': '/home/shawn/Programming/ai_stuff/llama.cpp/models/30B/ggml-model-q4_0.bin', 'choices': [{'text': 'Input: instruction\n Output: This read has a mutation. What is the metadata for this read? The read: GTCCTCCTCTTCTGAGGTTCAGGGAGTGGCCGCAGCGGAGGGGAAAAGCGAGACAGGTTAGTCTGGGAGCAGAGAGCCCGTGGCAGGAGCAGCCACCCAG\nInput: input\n Output: Chromsome position: chr1, Start position: 201369845, Reference Genome: hg38\nInput: output\n Output: the start pos from tsv is 201369845 the start pos from pileup is 201369835 the the gene is: TNNT2, the mutated base is: A\n\nInput: Reference Genome: hg38, Read: TAGAAAATGTGGATGAGACCTTCTGCATAGATAACGAAGCGCTATATGACATATGTTCCAGGACCCTAAAACTGCCCACACCCACCTATGGTGACCTGAA\nOutput:  Chromsome position: chr1, Start position: 201369845, Reference Genome: hg38', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 252, 'completion_tokens': 


llama_print_timings:        load time =  3721.41 ms
llama_print_timings:      sample time =    14.77 ms /    31 runs   (    0.48 ms per run)
llama_print_timings: prompt eval time = 137678.57 ms /   252 tokens (  546.34 ms per token)
llama_print_timings:        eval time = 27224.78 ms /    30 runs   (  907.49 ms per run)
llama_print_timings:       total time = 164922.61 ms


TypeError: can only concatenate str (not "dict") to str

# Fine tuning

In [22]:
modelPath='/home/shawn/datasets/LLMs/llama_7b/config.json'
memoryDir='/home/shawn/datasets'
trainingDataset=dnaset2.mutationDictionary 
#fineTuningDataset=dnaDataSet(modelPath=modelPath, memoryDir=memoryDir)
#fineTuningDataset.mutationDictionary = load_dataset(trainingDataset)

In [26]:
fineTuningDataset=dnaset2

In [27]:
fineTuningDataset

dnaDataSet object
 current model being used: False,
 mutationDictionary: {'Reference Genome: hg38, Read: GGACAAGGAGGCTGCACTCGGGACAGCAGGCAGCAGAAAGCACCACAGAAGGAAAGGCTGTACTACCGTCTTCGTCCTCTCTCCAGTCCTCCTCTTCTGA': 'the start pos from tsv is 201369845 the start pos from pileup is 201369750 the the gene is: TNNT2 and the mutated base is: A', 'Reference Genome: hg38, Read: ACAAGGAGGCTGCACTCGGGACAGCAGGCAGCAGAAAGCACCACAGAAGGAAAGGCTGTACTACCGTCTTCGTCCTCTCTCCAGTCCTCCTCTTCTGAGG': 'the start pos from tsv is 201369845 the start pos from pileup is 201369752 the the gene is: TNNT2 and the mutated base is: G', 'Reference Genome: hg38, Read: CAGAAAGCACCACAGAAGGAAAGGCTGTACTACCGTCTTCGTCCTCTCTCCAGTCCTCCTCTTCTGAGGTTCAGGGAGTGGCCGCAGCGGAGGGGAAAAG': 'the start pos from tsv is 201369845 the start pos from pileup is 201369783 the the gene is: TNNT2 and the mutated base is: T', 'Reference Genome: hg38, Read: ACAGAAGGAAAGGCTGTACTACCGTCTTCGTCCTCTCTCCAGTCCTCCTCTTCTGAGGTTCAGGGAGTGGCCGCAGCGGAGGGGAAAAGCGAGACAGGTT': 'the s

In [28]:
model = fineTuningDataset.finetune()

KeyError: 'train'

In [ ]:
## get the gvcf files
sav_dir = os.getcwd()
gvcf_dir = '/scratch/users/sschulz/pta_on_normal/gvcf'
download_log_dir='scratch/users/sschulz/pta_on_normal/gvcf/logs/'
os.chdir(gvcf_dir)

new_list = [chrom for sublist in [('MT', 'X', 'Y'), list(range(1,23))] for chrom in sublist]
new_list
def downloadEnsembleGVCFs(output_dir, download_log_dir):
    sav_dir = os.getcwd()
    os.chdir(output_dir)
    for i in [chrom for sublist in [('MT', 'X', 'Y'), list(range(1,23))] for chrom in sublist]:
        command_ending = str(chrom) + '.gvf.gz'
        print(f"sbatch -c 2 --mem=32G -p cgawad --out={download_log_dir} --wrap='wget https://ftp.ensembl.org/pub/release-109/variation/gvf/homo_sapiens/homo_sapiens_incl_consequences-chr{i}.gvf.gz'")
        !sbatch -c 2 --mem=32G -p cgawad --out=$download_log_dir --wrap=f"wget https://ftp.ensembl.org/pub/release-109/variation/gvf/homo_sapiens/homo_sapiens_incl_consequences-chr{i}.gvf.gz"
    os.chdir(sav_dir)
downloadEnsembleGVCFs(gvcf_dir, download_log_dir)
for i in ('MT', 'X', 'Y'):
    print(f"sbatch -c 2 --mem=32G -p cgawad --out='/scratch/users/sschulz/pta_on_normal/gvcf/logs/' --wrap='wget https://ftp.ensembl.org/pub/release-109/variation/gvf/homo_sapiens/homo_sapiens_incl_consequences-chr{i}.gvf.gz'")
     